# The complete simplex tableau method

## [Michel Bierlaire](https://people.epfl.ch/michel.bierlaire), EPFL.

This is Algorithm 16.5 in <a href="http://optimizationprinciplesalgorithms.com/">Bierlaire (2015) Optimization: principles and algorithms, EPFL Press.</a>

In [1]:
import numpy as np
import pandas as pd

We also import the scipy optimization library to compare the results.

In [2]:
import scipy.optimize as opt

First, the routine that performs one iteration of the simplex using the tableau.

In [3]:
def simplexTableau(tableau):
    """
    :param tableau: the first simplex tableau
    :type tableau: pandas dataframe
    
    :return: p, q, opt, bounded  where 
               - p is the column of the variable that must enter the basis, or None,
               - q is the row of the variable that must leave the basis, or None,
               - opt is True if the tableau is optimal (in this case, p and q are None)
               - bounded is True if basic direction is unbounded (in this case, p and q are None)
    :rtype: int, int, bool, bool
    """
    mtab, ntab = tableau.shape
    m = mtab - 1
    n = ntab - 1

    reducedCost = tableau[-1, : -1]
    # Identify the negative reduced costs
    negativeReducedCost = reducedCost < 0
    if not negativeReducedCost.any():
        # The tableau is optimal
        return None, None, True, True

    # In Python, True is larger than False. The next statement returns the 
    # index of a True entry in the array, that is the index of a negative reduced cost.
    # It is the index of the variable that will enter the basis.
    p = np.argmax(negativeReducedCost)

    # Calculate the maximum step that can be done along the basic direction d[p]
    xb = tableau[:-1, -1]
    minusd = tableau[:-1, p]
    steps = np.array([xb[k] / minusd[k] if minusd[k] > 0 else np.inf for k in range(m)])
    q = np.argmin(steps)            
    step = steps[q]

    if step == np.inf:
        # The tableau is unbounded
        return None, None, False, False
    else:
        return p, q, False, True

Then, the routine to pivot the tableau.

In [4]:
def pivoting(tableau, p, q):
    """
    :param tableau: valid simplex tableau
    :type tableau: numpy.array 2D
    
    :param p: columns of the pivot
    :type p: int
    
    :param q: row of the pivot
    :type q: int
    
    :return: pivoted tableau
    :rtype: numpy.array 2D
    """
    m, n = tableau.shape
    if q >= m:
        raise Exception(f'The row of the pivot ({q}) must be between 0 and {m - 1})')
    if p >= n:
        raise Exception(f'The column of the pivot ({p}) must be between 0 and {n - 1})')
    thepivot = tableau[q][p]
    if np.abs(thepivot) < np.finfo(float).eps:
        print(f'Tableau: {tableau})')
        print(f'Row {q} Col {p}')
        raise Exception(f'The pivot is too close to zero: {thepivot}')
    thepivotrow = tableau[q, :]
    newtableau = np.empty(tableau.shape)
    newtableau[q, :] = tableau[q, :] / thepivot
    for i in set(range(m)) - {q}:
        newtableau[i, :] = tableau[i, :] - tableau[i][p] * thepivotrow / thepivot
    return newtableau

The phase II algorithm.

In [5]:
def simplexAlgorithmTableau(tableau):
    """
    :param tableau: valid simplex tableau
    :type tableau: numpy.array 2D

    :return: tableau, optimal, unbounded, where tableau is the tableau from the last iteration,
                                          optimal is True if the last tableau is optimal,
                                          unbounded is True if the problem is unbounded.
    :rtype: numpy.array 2D, bool, bool
    """
    while True:
        colPivot, rowPivot, optimal, bounded = simplexTableau(tableau)
        if optimal:
            return tableau, True, False
        if not bounded:
            return tableau, False, True
        tableau = pivoting(tableau, colPivot, rowPivot)

The following routine identifies the row of the tableau corresponding to a given variable.

In [6]:
def getRow(tableau, index):
    """
    :param tableau: a valid simplex tableau.
    :type tableau: numpy.array 2D
    
    :param index: the index of the variable
    :type index: int
    
    :return: row index if the variable is basic, None otherwise
    :rtype: int
    """
    mtab, ntab = tableau.shape
    m = mtab - 1
    n = ntab - 1
    if index not in range(n):
         raise Exception(f'The index of the variable ({index}) must be between 0 and {n - 1})')
    column = tableau[:, index]
    rowIndex = None
    for j in range(mtab):
        if np.abs(column[j]) > np.sqrt(np.finfo(float).eps):
            # The entry is non zero
            if rowIndex is None and np.abs(column[j] - 1) <= np.finfo(float).eps:
                # The entry is one, and the index has not been found yet.
                rowIndex = j
            else:
                return None
    return rowIndex

Finally, the complete simplex algorithm.

In [7]:
def simplex(A, b, c):
    """
    :param A: m x n matrix of the constraints
    :type A: numpy.array 2D
    
    :param b: m vector, left-hand side of the constraints.
    :type b: numpy.array 1D
    
    :param c: n vector, coefficients of the objective function 
    :type c: numpy.array 1D

    :return: tableau, unbounded, infeasible
                where  - tableau is the tableau from the last iteration,
                       - unbounded is True if the problem is unbounded,
                       - infeasible is True if the problem is infeasible,
    :rtype: numpy.array 2D, bool, bool
    """

    m, n = A.shape
    if b.shape[0] != m:
        raise Exception(f'Incompatible sizes: A is {m}x{n}, b is of length {b.shape[0]}, and should be {m}')
    if c.shape[0] != n:
        raise Exception(f'Incompatible sizes: A is {m}x{n}, c is of length {c.shape[0]}, and should be {n}')
    
    # All elements of b must be non negative.
    negativeb = np.argwhere(b < 0)
    for i in negativeb:
        A[i, :] = - A[i, :]
        b[i] = - b[i]

    # First tableau for the auxiliary problem
    tableau = np.empty((m + 1, n + m + 1))
    tableau[:m, :n] = A
    tableau[:m, n:n + m] = np.eye(m)
    tableau[:m, n + m:n + m + 1] = b.reshape(m, 1)
    # The last row 
    tableau[-1, :n] = np.array([-np.sum(tableau[:m, k]) for k in range(n)]).copy()
    tableau[-1, n:n + m] = 0.0
    tableau[-1, -1] = -np.sum(tableau[:m, -1])
    
    
    # Solve the auxiliary problem
    phaseOneTableau, optimal, unbounded = simplexAlgorithmTableau(tableau)

    if unbounded:
        print('UNBOUNDED')
        return tableau, True, False

    if phaseOneTableau[-1, -1]  < -np.sqrt(np.finfo(float).eps):
        # Infeasible problem
        print('INFEASIBLE')
        return phaseOneTableau, False, True
 
    # Remove the auxiliary variables from the basis
    clean = False
    rowsToRemove = []
    basicRows = np.array([getRow(phaseOneTableau,k) for k in range(m + n)])
    
    while not clean:
        basicIndices = set(np.where(basicRows != None)[0])
        # Check if some auxiliary variables are in the basis
        tobeCleaned = set(basicIndices).intersection(set(range(n,n+m))) 
        if tobeCleaned == set():
            clean = True
        else:
            auxiliaryColumn = tobeCleaned.pop()
            rowpivotIndex = basicRows[auxiliaryColumn]
            rowpivot = phaseOneTableau[rowpivotIndex,:]
            originalNonbasic = list(set(range(n)) - set(basicIndices))
            nonzerosPivots = abs(rowpivot[originalNonbasic]) > np.finfo(float).eps
            if nonzerosPivots.any():
                # It is possible to pivot
                colpivot = originalNonbasic[np.argmax(nonzerosPivots)]
                phaseOneTableau = pivoting(phaseOneTableau, colpivot, rowpivotIndex)      
                basicRows[colpivot] = rowpivotIndex
                basicRows[auxiliaryColumn] = None
            else:
                # It is not possible to pivot. There is a redundant 
                # constraint to be removed.
                rowsToRemove.append(rowpivotIndex)
                phaseOneTableau = np.delete(phaseOneTableau,rowsToRemove,0)
                basicRows = np.array([getRow(phaseOneTableau,k) for k in range(m + n)])

    # Delete columns
    startPhaseTwo = np.delete(phaseOneTableau, range(n, n + m), 1)
    m -= len(rowsToRemove)
    basicRows = np.array([getRow(startPhaseTwo, k) for k in range(n)])
    
    # Calculate last row
    
    basicIndices = list(np.where(basicRows != None)[0])
    nonbasicIndices = list(np.where(basicRows == None)[0])
    cb = c[basicIndices]
    for k in nonbasicIndices:
        startPhaseTwo[-1, k] = c[k] - np.array([c[j] * startPhaseTwo[basicRows[j],k] for j in basicIndices]).sum()
    startPhaseTwo[-1, -1] = - np.array([c[j] * startPhaseTwo[basicRows[j],-1] for j in basicIndices]).sum()

    # Phase II
    
    phaseTwoTableau, optimal, unbounded = simplexAlgorithmTableau(startPhaseTwo)
    return phaseTwoTableau, unbounded, False

A couple of routines to format the output.

In [8]:
def prettyTableau(tableau):
    m, n = tableau.shape
    s = ''
    for i in range(m - 1):
        formattedRow = ['{:6.2g}' for k in tableau[i, :-1]]
        s += '\t'.join(formattedRow).format(*tableau[i, :-1])
        s += f'|{tableau[i, -1]:6.2f}'
        s += '\n'
    for i in range(n):
        s += '------\t'
    s += '\n'
    formattedRow = ['{:6.2g}' for k in tableau[m - 1, :-1]]
    s += '\t'.join(formattedRow).format(*tableau[m - 1, :-1])
    s += f'|{tableau[m - 1,- 1]:6.2f}'
    s += '\n'
    
    return s

In [9]:
def printResults(res):
    finalTableau, unbounded, infeasible = res
    if unbounded:
        print('Unbounded problem')
    elif infeasible:
        print('Infeasible problem')
    else:
        print(prettyTableau(finalTableau))
        mtab, ntab = finalTableau.shape
        m = mtab - 1
        n = ntab - 1
        basicRows = [getRow(finalTableau, k) for k in range(n)]
        solution = [finalTableau[basicRows[k], -1] if basicRows[k] is not None else 0 for k in range(n)] 
        copt = -finalTableau[-1, -1]
        print(f'Optimal solution: {solution}')
        print(f'Optimal value: {copt}')

# Example 1

\\[ A = \left(\begin{array}{cccc} 1 & 2 & 3 & 0 \\ -1 & 2 & 6 & 0 \\ 0 & 4 & 9& 0 \\ 0 & 0 & 3 & 1 \\ \end{array} \right), \qquad b = \left( \begin{array}{c} 3 \\ 2 \\ 5 \\ 1 \end{array} \right), \qquad c = \left( \begin{array}{c} 1 \\ 1 \\ 1 \\ 0\end{array} \right). \\]

In [10]:
A = np.array([[1, 2, 3, 0], [-1, 2, 6, 0], [0, 4, 9, 0], [0, 0, 3, 1]])
b = np.array([3, 2, 5, 1])
c = np.array([1, 1, 1, 0])

In [11]:
res = simplex(A, b, c)
printResults(res)

     1	     0	  -1.5	     0|  0.50
     0	     1	   2.2	     0|  1.25
     0	     0	     3	     1|  1.00
------	------	------	------	------	
     0	     0	  0.25	     0| -1.75

Optimal solution: [0.5, 1.25, 0, 1.0]
Optimal value: 1.75


We compare the results with scipy

In [12]:
sc = opt.linprog(c, A_eq = A, b_eq = b )

<ipython-input-12-b2dbb1be18ba>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  sc = opt.linprog(c, A_eq = A, b_eq = b )


In [13]:
print(sc)

     con: array([-9.61007451e-09, -1.60167883e-08, -2.56268633e-08, -9.61007363e-09])
     fun: 1.7500000032699403
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([4.99999997e-01, 1.25000001e+00, 2.66325791e-10, 1.00000001e+00])


# Example 2

\\[ A = \left(\begin{array}{cccc} 1 & 2 & 3 & 0 \\ -1 & 2 & 6 & 0 \\ 0 & 4 & 9 & 0 \\ 0 & 4 & 9 & 0 \\ 0 & 0 & 3 & 1  \end{array} \right), \qquad b = \left( \begin{array}{c} 3 \\ 2 \\ 5 \\ 5\\ 1 \end{array} \right), \qquad c = \left( \begin{array}{c} 1 \\ 1 \\ 1 \\ 0\end{array} \right). \\]

In [14]:
A = np.array([[1, 2, 3, 0], [-1, 2, 6, 0], [0, 4, 9, 0], [0, 4, 9, 0], [0, 0, 3, 1]])
b = np.array([3, 2, 5, 5, 1])
c = np.array([1, 1, 1, 0])

In [15]:
res = simplex(A, b, c)
printResults(res)

     1	     0	  -1.5	     0|  0.50
     0	     1	   2.2	     0|  1.25
     0	     0	     3	     1|  1.00
------	------	------	------	------	
     0	     0	  0.25	     0| -1.75

Optimal solution: [0.5, 1.25, 0, 1.0]
Optimal value: 1.75


We compare the results with scipy

In [16]:
sc = opt.linprog(c, A_eq = A, b_eq = b )
print(sc)

     con: array([-9.61007451e-09, -1.60167883e-08, -2.56268633e-08, -2.56268633e-08,
       -9.61007363e-09])
     fun: 1.7500000032699403
 message: 'Optimization terminated successfully.'
     nit: 4
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([4.99999997e-01, 1.25000001e+00, 2.66325791e-10, 1.00000001e+00])


<ipython-input-16-da9fd6071a23>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  sc = opt.linprog(c, A_eq = A, b_eq = b )


# Example 3

\\[ A = \left(\begin{array}{cccc} 1 & -1 & 1 & 0 \\ -1 & 1 & 0 & 1  \end{array} \right), \qquad b = \left( \begin{array}{c} -2 \\ -1 \end{array} \right), \qquad c = \left( \begin{array}{r} -4 \\ 2 \\ 0 \\ 0\end{array} \right). \\]

In [17]:
A = np.array([[1, -1, 1, 0], [-1, 1, 0, 1]])
b = np.array([-2, -1])
c = np.array([-4, 2, 0, 0])

In [18]:
res = simplex(A, b, c)
printResults(res)

INFEASIBLE
Infeasible problem


We compare the results with scipy

In [19]:
sc = opt.linprog(c, A_eq = A, b_eq = b )
print(sc)

     con: array([9.55219364, 6.33930206])
     fun: -12402393951.668787
 message: 'The algorithm terminated successfully and determined that the problem is infeasible.'
     nit: 4
   slack: array([], dtype=float64)
  status: 2
 success: False
       x: array([6.20119697e+09, 6.20119697e+09, 3.16055006e+00, 9.73094586e+00])


# Example 4

\\[ A = \left(\begin{array}{ccccc} 1 & 3 & 0 & 4 & 1 \\ 1 & 2 & 0 & -3 & 1 \\ -1 & -4 & 3 & 0 & 0 \end{array} \right), \qquad b = \left( \begin{array}{c} 2 \\ 2 \\ 1 \end{array} \right), \qquad c = \left( \begin{array}{r} 2 \\ 3 \\ 3 \\ 1 \\ -2\end{array} \right). \\]

In [20]:
A = np.array([[1, 3, 0, 4, 1], [1, 2, 0, -3, 1], [-1, -4, 3, 0, 0]])
b = np.array([2, 2, 1])
c = np.array([2, 3, 3, 1, -2])

In [21]:
res = simplex(A, b, c)
printResults(res)

     1	   2.4	     0	     0	     1|  2.00
     0	  0.14	     0	     1	     0|  0.00
 -0.33	  -1.3	     1	     0	     0|  0.33
------	------	------	------	------	------	
     5	    12	     0	     0	     0|  3.00

Optimal solution: [0, 0, 0.33333333333333337, 0.0, 2.0]
Optimal value: -3.0


We compare the results with scipy

In [22]:
sc = opt.linprog(c, A_eq = A, b_eq = b )
print(sc)

     con: array([-4.15534274e-12,  5.93969318e-13,  1.78124182e-12])
     fun: -2.9999999999980593
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([8.22979221e-13, 1.54636594e-13, 3.33333333e-01, 6.56391777e-13,
       2.00000000e+00])


# Example 5

\\[ A = \left(\begin{array}{ccccc} 1 & 2 & 1 & 0 & 0 \\ -3 & 2 & 0 & 1 & 0 \\ -2 & -3 & 0 & 0 & 1\end{array} \right), \qquad b = \left( \begin{array}{c} 31 \\ 5 \\ -1 \end{array} \right), \qquad c = \left( \begin{array}{r} -9 \\ -4 \\ 0 \\ 0 \\ 0\end{array} \right). \\]

In [23]:
A = np.array([[1, 2, 1, 0, 0], [-3, 2, 0, 1, 0], [-2, -3, 0, 0, 1]])
b = np.array([31, 5, -1])
c = np.array([-9, -4, 0, 0, 0])

In [24]:
res = simplex(A, b, c)
printResults(res)

     1	     2	     1	     0	     0| 31.00
     0	     1	     2	     0	     1| 61.00
     0	     8	     3	     1	     0| 98.00
------	------	------	------	------	------	
     0	    14	     9	     0	     0|279.00

Optimal solution: [31.0, 0, 0, 98.0, 61.0]
Optimal value: -279.0


We compare the results with scipy

In [25]:
sc = opt.linprog(c, A_eq = A, b_eq = b )
print(sc)

     con: array([ 7.55616760e-08,  1.39929881e-08, -8.39573744e-09])
     fun: -278.99999910864204
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([3.09999999e+01, 3.64640767e-09, 1.78059094e-08, 9.79999997e+01,
       6.09999998e+01])


# Example 6

\\[ A = \left(\begin{array}{cc} 1 & 0\end{array} \right), \qquad b = \left( \begin{array}{c} 1 \end{array} \right), \qquad c = \left( \begin{array}{r}3 \\ -2\end{array} \right). \\]

In [26]:
A = np.array([[1, 0]])
b = np.array([1])
c = np.array([3, -2])

In [27]:
res = simplex(A, b, c)
printResults(res)

Unbounded problem


We compare the results with scipy

In [28]:
sc = opt.linprog(c, A_eq = A, b_eq = b )
print(sc)

     con: array([nan])
     fun: -inf
 message: 'If feasible, the problem is (trivially) unbounded due  to a zero column in the constraint matrices. If you wish to check whether the problem is infeasible, turn presolve off.'
     nit: 0
   slack: array([], dtype=float64)
  status: 3
 success: False
       x: array([ 0., inf])


# Example 7

In [29]:
A = np.array([[-1,  0, -1,  0,  0,  0],
              [ 1,  1,  0,  0,  0,  0],
              [ 0, -1,  0,  0, -1,  0],
              [ 0,  0,  0,  0,  1,  1],
              [ 0,  0,  0, -1,  0, -1],
              [ 0,  0,  1,  1,  0,  0]
             ])
b = np.array([-30, 40, -40, 20, -30, 40])
c = np.array([0, 0.5, 0, 1, 0, 0.8])


In [30]:
np.sum(A,axis=0)

array([0, 0, 0, 0, 0, 0])

In [31]:
res = simplex(A, b, c)
printResults(res)

     0	     0	     1	     0	     0	    -1| 10.00
     0	     1	     0	     0	     0	    -1| 20.00
     0	     0	     0	     0	     1	     1| 20.00
     1	     0	     0	     0	     0	     1| 20.00
     0	     0	     0	     1	     0	     1| 30.00
------	------	------	------	------	------	------	
     0	     0	     0	     0	     0	   0.3|-40.00

Optimal solution: [20.0, 20.0, 10.0, 30.0, 20.0, 0]
Optimal value: 40.0


We compare the results with scipy

In [32]:
sc = opt.linprog(c, A_eq = A, b_eq = b )
print(sc)

     con: array([6.73352929e-10, 9.13900067e-10, 9.13956910e-10, 4.32898162e-10,
       6.73399114e-10, 9.13914278e-10])
     fun: 39.999999999086995
 message: 'Optimization terminated successfully.'
     nit: 5
   slack: array([], dtype=float64)
  status: 0
 success: True
       x: array([2.0000000e+01, 2.0000000e+01, 1.0000000e+01, 3.0000000e+01,
       2.0000000e+01, 3.0868254e-12])


<ipython-input-32-da9fd6071a23>:1: OptimizeWarning: A_eq does not appear to be of full row rank. To improve performance, check the problem formulation for redundant equality constraints.
  sc = opt.linprog(c, A_eq = A, b_eq = b )
